In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from keras.preprocessing import image

import warnings
warnings.filterwarnings('ignore')

In [3]:
train_image='D:/Documents/Capstone/Project -2/Dataset/bacterial_leaf_blight_Vs_non-bacterial_leaf_blight/train'

In [4]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 zoom_range=0.2,
                                 shear_range=0.1,
                                vertical_flip=False)
training_set=train_datagen.flow_from_directory(train_image,
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)

Found 2846 images belonging to 2 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255,
                                 zoom_range=0.2)

test_set=train_datagen.flow_from_directory('D:/Documents/Capstone/Project -2/Dataset/bacterial_leaf_blight_Vs_non-bacterial_leaf_blight/test',
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)

Found 422 images belonging to 2 classes.


In [6]:
base_model = tf.keras.applications.InceptionResNetV2(
                     include_top=False,
                     weights='imagenet',
                     input_shape=(224,224,3)
                     )
 
base_model.trainable=False
 
model = tf.keras.Sequential([
        base_model,  
        tf.keras.layers.BatchNormalization(renorm=True),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

In [7]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 5, 5, 1536)       54336736  
 onal)                                                           
                                                                 
 batch_normalization_203 (Ba  (None, 5, 5, 1536)       10752     
 tchNormalization)                                               
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               786944    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                        

In [8]:
early = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=0.001, restore_best_weights=True)

In [9]:
batch_size=32
STEP_SIZE_TRAIN = training_set.n//training_set.batch_size
STEP_SIZE_VALID = test_set.n//test_set.batch_size

# fit model
history = model.fit(training_set, steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_set,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10,
                    callbacks=[early])

Epoch 1/10
569/569 [==============================] - 2322s 4s/step - loss: 0.5107 - accuracy: 0.7645 - val_loss: 0.4091 - val_accuracy: 0.8238
Epoch 2/10
569/569 [==============================] - 2896s 5s/step - loss: 0.3731 - accuracy: 0.8416 - val_loss: 0.3343 - val_accuracy: 0.8476
Epoch 3/10
569/569 [==============================] - 3079s 5s/step - loss: 0.3002 - accuracy: 0.8814 - val_loss: 0.2715 - val_accuracy: 0.8881
Epoch 4/10
569/569 [==============================] - 2550s 4s/step - loss: 0.2644 - accuracy: 0.9000 - val_loss: 0.2353 - val_accuracy: 0.9143
Epoch 5/10
569/569 [==============================] - 2718s 5s/step - loss: 0.2378 - accuracy: 0.9155 - val_loss: 0.2722 - val_accuracy: 0.9190
Epoch 6/10
569/569 [==============================] - 2440s 4s/step - loss: 0.2103 - accuracy: 0.9194 - val_loss: 0.1797 - val_accuracy: 0.9167
Epoch 7/10
569/569 [==============================] - 1984s 3s/step - loss: 0.2037 - accuracy: 0.9166 - val_loss: 0.1962 - val_accuracy: